In [1]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, LabelBinarizer
from sklearn.model_selection import StratifiedKFold
import pickle
from random import choice
import os.path

import tensorflow as tf
import numpy as np

In [2]:
tf.enable_eager_execution()

#### Import model, training function 

In [3]:
from annsa.model_classes import (dnn_model_features,
                                 DNN,
                                 save_model,
                                 train_earlystop)

#### Load Dataset

In [4]:
all_spectra = np.load('./hyperparameter_search_datasets/spectra_SBratio13_CalRange0911_3625.npy')
all_keys = np.load('./hyperparameter_search_datasets/keys_SBratio13_CalRange0911_3625.npy')

skf = StratifiedKFold(n_splits=5, random_state=5)

#### Define scaling pipeline, key binarizer

In [5]:
scaler = make_pipeline(FunctionTransformer(np.log1p, validate=True))
mlb = LabelBinarizer()

#### Search through hyperparameters

In [6]:
number_hyperparameters_to_search = 256
earlystop_errors_test = []
model_id = 'vanilla_dnn'
model_features_class = dnn_model_features
model_class = DNN

In [7]:
file_path = './hyperparameter-search-results/final_test_errors_'+model_id+'.npy'
if os.path.exists(file_path):
    earlystop_errors_test = np.load(file_path)
    start_iteration = earlystop_errors_test.shape[0]
else:
    start_iteration = 0

In [ ]:
for hyperparameter_index in range(start_iteration,
                                  number_hyperparameters_to_search):
    print("\nSearching hyperparameter combination number "+
          str(hyperparameter_index))

    number_layers = np.random.randint(1, 4)
    dense_nodes = (10**np.random.uniform(1, 3, number_layers)).astype('int')
    dense_nodes = np.sort(dense_nodes)
    dense_nodes = np.flipud(dense_nodes)

    model_features = model_features_class(
        learining_rate=10**np.random.uniform(-4,-1),
        l2_regularization_scale=10**np.random.uniform(-2,1),
        dropout_probability=np.random.uniform(0,1),
        batch_size=2**np.random.randint(4,10),
        output_size=29,
        dense_nodes=dense_nodes,
        activation_function=choice([tf.nn.relu,tf.nn.tanh,tf.nn.sigmoid]),
        scaler=scaler)

    with open('./hyperparameter-search-results/'+model_id+'_'+
              str(hyperparameter_index)+'_features', 'wb+') as f:
        pickle.dump(model_features,f)

    optimizer = tf.train.AdamOptimizer(model_features.learining_rate)
    model = model_class(model_features)

    k_folds_errors = []
    for train_index, test_index in skf.split(all_spectra, all_keys):
        scaler.fit(all_spectra[train_index])
        
        # reset model on each iteration
        model = model_class(model_features)
        all_keys_binarized = mlb.fit_transform(all_keys.reshape([all_keys.shape[0],1]))

        costfunction_errors_tmp, earlystop_errors_tmp = train_earlystop(
            training_data=all_spectra[train_index],
            training_keys=all_keys_binarized[train_index],
            testing_data=all_spectra[test_index],
            testing_keys=all_keys_binarized[test_index],
            model=model,
            optimizer=optimizer,
            num_epochs=200,
            obj_cost=model.cross_entropy,
            earlystop_cost_fn=model.f1_error,
            earlystop_patience=20,
            verbose=True,
            fit_batch_verbose=5,
            data_augmentation=model.default_data_augmentation)
        k_folds_errors.append(earlystop_errors_tmp)

    earlystop_errors = np.average(k_folds_errors)
    earlystop_errors_test.append(earlystop_errors)
    np.save('./hyperparameter-search-results/final_test_errors_'+
            model_id, earlystop_errors_test)


Searching hyperparameter combination number 21
Epoch 5: CostFunc loss: 3.52 3.52, EarlyStop loss: 0.97 0.97
Epoch 10: CostFunc loss: 3.38 3.38, EarlyStop loss: 0.97 0.97
Epoch 15: CostFunc loss: 3.37 3.37, EarlyStop loss: 0.97 0.97
Epoch 20: CostFunc loss: 3.37 3.37, EarlyStop loss: 0.97 0.97
Test error at early stop: Objectives fctn: 4.99 Early stop fctn: 4.99
Epoch 5: CostFunc loss: 3.70 3.70, EarlyStop loss: 0.97 0.97
Epoch 10: CostFunc loss: 3.38 3.38, EarlyStop loss: 0.97 0.97
Epoch 15: CostFunc loss: 3.38 3.38, EarlyStop loss: 0.97 0.97
Epoch 20: CostFunc loss: 3.38 3.38, EarlyStop loss: 0.97 0.97
Test error at early stop: Objectives fctn: 7.00 Early stop fctn: 7.00
Epoch 5: CostFunc loss: 3.93 3.93, EarlyStop loss: 0.97 0.97
Epoch 10: CostFunc loss: 3.38 3.38, EarlyStop loss: 0.97 0.97
Epoch 15: CostFunc loss: 3.38 3.38, EarlyStop loss: 0.97 0.97
Epoch 20: CostFunc loss: 3.37 3.37, EarlyStop loss: 0.97 0.97
Test error at early stop: Objectives fctn: 9.05 Early stop fctn: 9.05
E